In [1]:
# import libraries

import pandas as pd


import numpy as np

import warnings
warnings.filterwarnings ('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import seaborn as sns

import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim

In [2]:
data_prior3_ori = pd.read_csv('../Proyecto_Arraigo_Recomender_System/data/Urbanitas_prior3.csv')


data_prior3 = data_prior3_ori.copy()

In [3]:
data_prior3.shape

(10618, 23)

In [4]:
data_prior3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10618 entries, 0 to 10617
Data columns (total 23 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Unnamed: 0                          10618 non-null  int64 
 1   id                                  10618 non-null  object
 2   fecha                               10618 non-null  object
 3   pais_de_residencia                  10590 non-null  object
 4   provincia_de_espana                 10618 non-null  object
 5   situacion_en_espana                 10571 non-null  object
 6   profesion                           10515 non-null  object
 7   autonomia_economica                 8407 non-null   object
 8   familia_edades                      8964 non-null   object
 9   coche_propio                        10618 non-null  object
 10  carnet_coche                        10618 non-null  object
 11  mascota                             10618 non-null  ob

In [5]:
porcentaje_nulos = data_prior3.isnull().sum() / data_prior3.count() * 100
porcentaje_nulos.sort_values(ascending=False)

autonomia_economica                   26.299512
familia_edades                        18.451584
profesion                              0.979553
primera_segunda_vivienda               0.587344
situacion_en_espana                    0.444613
presupuesto_alquiler                   0.387634
pais_de_residencia                     0.264400
Unnamed: 0                             0.000000
predisposicion_pueblo                  0.000000
Otros                                  0.000000
sector_agricultura_ganaderia_pesca     0.000000
sector_construccion                    0.000000
sector_industria                       0.000000
sector_servicios                       0.000000
numero_familiares                      0.000000
mascota                                0.000000
zona_interes                           0.000000
id                                     0.000000
carnet_coche                           0.000000
coche_propio                           0.000000
provincia_de_espana                    0

BORRADO DE COLUMNAS INCONSISTENTES/IRRELEVANTES

In [6]:
data_prior3 = data_prior3.drop(labels=["Unnamed: 0",
                                       'id',
                                       'nan_filas',
                                       'profesion',
                                       'familia_edades'],
                                        axis=1)


In [7]:
data_prior3.head(1)

,fecha,pais_de_residencia,provincia_de_espana,situacion_en_espana,autonomia_economica,coche_propio,carnet_coche,mascota,presupuesto_alquiler,primera_segunda_vivienda,zona_interes,predisposicion_pueblo,numero_familiares,sector_servicios,sector_industria,sector_construccion,sector_agricultura_ganaderia_pesca,Otros
0,2023-03-01,España,Cantabria,Tengo nacionalidad española / de un país de la...,NaN,No,Si,No,NaN,NaN,Indiferente,Aún me lo estoy pensando,3,0,0,0,0,0


In [8]:
mask = data_prior3.apply(lambda row: row.astype(str).str.contains('\?').any(), axis=1)
result = data_prior3.loc[mask]

result

,fecha,pais_de_residencia,provincia_de_espana,situacion_en_espana,autonomia_economica,coche_propio,carnet_coche,mascota,presupuesto_alquiler,primera_segunda_vivienda,zona_interes,predisposicion_pueblo,numero_familiares,sector_servicios,sector_industria,sector_construccion,sector_agricultura_ganaderia_pesca,Otros
72,2020-12-07,Latino América,Ninguna,No estoy en ninguna de esas situaciones,6 meses,Si,Si,No,Hasta 300€,Primera residencia,? ?????????,1 mes,2,0,0,0,0,1
5290,2021-11-04,Otro,Ninguna,No estoy en ninguna de esas situaciones,???? ?? ???,Si,Si,Si,Hasta 300€,Primera residencia,??? ????,Aún me lo estoy pensando,3,0,0,0,0,1
5847,2022-01-01,Otro,Ninguna,No estoy en ninguna de esas situaciones,3 meses,Si,Si,No,Hasta 300€,Primera residencia,? ?????????,1 mes,6,0,0,0,0,0
9314,2022-09-09,Otro,Ninguna,No estoy en ninguna de esas situaciones,NaN,Si,Si,Si,Hasta 300€,Primera residencia,?????? ????????,1 mes,3,0,0,0,0,1
10256,2022-12-26,España,????????,Tengo nacionalidad española / de un país de la...,NaN,Si,Si,No,Hasta 300€,Primera residencia,Indiferente,6 meses,2,0,0,0,0,0


In [9]:
data_prior3 = data_prior3.drop(result.index)

BORRADO DE NULOS (SALVO AUTONOMÍA ECONÓMICA)

In [10]:
data_prior3 = data_prior3.dropna(subset=[col for col in data_prior3.columns if col != 'autonomia_economica'])

CONVERSIÓN DE TODAS LAS COLUMNAS CATEGÓRICAS A NUMÉRICAS

In [11]:
#Empezaremos por las columnas más sencillas, pasándolas a booleano (éxito, no éxito).

In [12]:
data_prior3['carnet_coche']= data_prior3.carnet_coche.replace('Si', 1).replace('No', 0).astype(int)

In [13]:
data_prior3['coche_propio']= data_prior3.coche_propio.replace('Si', 1).replace('No', 0).astype(int)

In [14]:
data_prior3['mascota'] = data_prior3.mascota.replace('Si', 1).replace('No', 0).astype(int)

In [15]:
data_prior3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10449 entries, 1 to 10617
Data columns (total 18 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   fecha                               10449 non-null  object
 1   pais_de_residencia                  10449 non-null  object
 2   provincia_de_espana                 10449 non-null  object
 3   situacion_en_espana                 10449 non-null  object
 4   autonomia_economica                 8313 non-null   object
 5   coche_propio                        10449 non-null  int32 
 6   carnet_coche                        10449 non-null  int32 
 7   mascota                             10449 non-null  int32 
 8   presupuesto_alquiler                10449 non-null  object
 9   primera_segunda_vivienda            10449 non-null  object
 10  zona_interes                        10449 non-null  object
 11  predisposicion_pueblo               10449 non-null  ob

In [17]:
data_prior3.predisposicion_pueblo .unique() #en cuanto tiempo estarían preparados para mudarse a un pueblo

array(['3 meses', 'Aún me lo estoy pensando', '6 meses', '1 mes'],
      dtype=object)

In [18]:
data_prior3 = pd.get_dummies(data_prior3, columns=['predisposicion_pueblo'])

In [21]:
data_prior3.shape

(10449, 21)

In [23]:
#Columna de fecha: como nos puede interesar la estacionalidad de demanda y la tendencia, haremos lo siguiente:

# Convertimos la columna de fecha a tipo datetime
data_prior3['fecha'] = pd.to_datetime(data_prior3['fecha'])

# Creamos nuevas columnas numéricas para el año, mes y día
data_prior3['year'] = data_prior3['fecha'].dt.year
data_prior3['month'] = data_prior3['fecha'].dt.month
data_prior3['day'] = data_prior3['fecha'].dt.day #esta incluso nos la podríamos cargar, pero la dejamos de momento

In [24]:
data_prior3.shape

(10449, 24)

In [25]:
#columna pais_de_residencia

data_prior3.pais_de_residencia.unique()

array(['España', 'Latino América', 'Otro',
       'Resto de Europa (dentro de UE)'], dtype=object)

In [26]:
data_prior3 = pd.get_dummies(data_prior3, columns=['pais_de_residencia'])

In [27]:
#columna provincia

data_prior3.provincia_de_espana.unique()

array(['Andalucía', 'Madrid', 'Valencia', 'Ninguna', 'Baleares',
       'Navarra', 'Comunitat Valenciana', 'Pontevedra', 'Barcelona',
       'Toledo', 'Alicante', 'Canarias', 'Aragón', 'Extremadura',
       'Euskadi', 'Cataluña', 'Castilla y León', 'Galicia', 'Vizcaya',
       'Sevilla', 'Murcia', 'Cádiz', 'Castilla-La Mancha', 'Cantabria',
       'Región de Murcia', 'Tarragona', 'La Rioja', 'Córdoba', 'A Coruña',
       'Ciudad Real', 'Girona', 'Albacete', 'Castellón', 'Málaga',
       'Cáceres', 'Asturias', 'Guadalajara', 'Plazas de Soberanía',
       'Granada', 'Zaragoza', 'Almería', 'Segovia', 'Huelva',
       'Valladolid', 'Las Palmas', 'León', 'Santa Cruz de Tenerife',
       'Lérida', 'Gipuzkoa', 'Badajoz', 'Huesca', 'Ceuta y Melilla',
       'Cuenca', 'Jaén', 'Salamanca', 'Burgos', 'Lleida', 'Álava',
       'Zamora', 'Palencia', 'Ourense', 'Lugo', 'Ávila', 'Teruel',
       'Soria'], dtype=object)

In [31]:
#como get dummies nos crearía muchas columnas, dividiremos las provincias por 4 regiones de España

prov_norte = ['Galicia', 'Asturias', 'Cantabria', 'País Vasco', 'Navarra', 'La Rioja','Pontevedra','Euskadi','Vizcaya','A Coruña','Gipuzkoa''Ourense', 'Lugo','La Rioja', ]
prov_sur = ['Andalucía','Cádiz','Córdoba','Málaga','Sevilla', 'Granada', 'Almería', 'Huelva', 'Ceuta y Melilla','Canarias','Las Palmas','Santa Cruz de Tenerife']
prov_este = ['Cataluña', 'Baleares','Valencia', 'Murcia','Alicante', 'Comunitat Valenciana','Región de Murcia','Castellón','Lérida','Lleida','Girona','Tarragona', 'Barcelona']
prov_centro = ['Castilla y León', 'Castilla-La Mancha', 'Aragón', 'Toledo','Extremadura', 'Ciudad Real','Albacete','Cáceres','Guadalajara','Zaragoza','Segovia','Valladolid','León','Badajoz', 'Huesca','Cuenca', 'Jaén','Salamanca', 'Burgos','Zamora', 'Palencia','Ávila', 'Teruel','Soria']
prov_madrid = ['Madrid']
prov_fuera_espana = ['Ninguna','Plazas de Soberanía']
